# Process the TeleQnA Dataset to create and save train datasets

Processing TeleQnA dataset to produces datasets to fine tunne a model and then test it.

# Dataset TeleQnA
We will load json files that we generated and create dataset to fine-tunning the model.

## Questions Release 17

## Dataset with 4000 questions

### Questions Release 17

In [ ]:
import json

# Path to the TeleQnA processed question in JSON file with only Rel 17...
rel17_questions_path = r"../files/rel17_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_questions_path, "r", encoding="utf-8") as file:
    rel17_questions = json.load(file)
print(len(rel17_questions))

# Path to the TeleQnA processed question in JSON file with only Rel 17 and 100 questions...
rel17_100_questions_path = r"../files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_100_questions_path, "r", encoding="utf-8") as file:
    rel17_100_questions = json.load(file)
print(len(rel17_100_questions))

rel17_other_questions = [q for q in rel17_questions if q not in rel17_100_questions]
print(len(rel17_other_questions))

rel17_other_questions_length = 500
rel17_other_questions = rel17_other_questions[:rel17_other_questions_length]
print(len(rel17_other_questions))
rel17_other_questions[0]

### Questions without Rel 17 and 18

In [ ]:
# Path to the TeleQnA processed question in JSON file with questions without Rel 17 and 18...
no_rel_17_18_path_questions = r"../files/no_rel_17_18_questions.json"

# Load the TeleQnA data...
with open(no_rel_17_18_path_questions, "r", encoding="utf-8") as file:
    no_rel_17_18_questions = json.load(file)
print(len(no_rel_17_18_questions))

no_rel_17_18_questions_length = 3500
no_rel_17_18_questions = no_rel_17_18_questions[:no_rel_17_18_questions_length]
print(len(no_rel_17_18_questions))
no_rel_17_18_questions[0]

### Train data

In [ ]:
train_questions = rel17_other_questions + no_rel_17_18_questions
print(len(train_questions))
train_questions[0]

We create two datasets, one with no options and half of questions and another wit options and the other half.

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions and explanations
data = []

half_questions = len(train_questions)//2

# Fill the dataset with (question, explanation) pairs
for item in train_questions[:half_questions]:

    human_value = (
        f"{item['question']}"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"{item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # For the question
        {'from': 'gpt', 'value': gpt_value}  # For the explanation
    ]

    data.append(pair)  # Add the pair to the dataset

data_no_options = data
print(data_no_options[0])

In [ ]:
from datasets import Dataset

# Structure to store pairs of questions, options, answers, and explanations
data = []

# Fill the dataset with (question + options, answer + explanation) pairs
for item in train_questions[half_questions:]:

    # Extract options
    options = [f"{key}: {value}" for key, value in item.items() if 'option' in key]
    # Combine the question and options
    human_value = (
        f"Question: {item['question']}\n"
        f"Options:\n" + "\n".join(options) + "\n"
    )

    # Combine the answer and explanation
    gpt_value = (
        f"Answer: {item['answer']}\n"
        f"Explanation: {item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # Question with options
        {'from': 'gpt', 'value': gpt_value}       # Answer with explanation
    ]

    data.append(pair)  # Add the pair to the dataset

# Create the dataset using Hugging Face
data_options = data
print(data_options[0])

Then we join these datasets and shuffle randomly.

In [ ]:
import random

data_total = data_no_options + data_options
# Shuffle the combined data
random.shuffle(data_total)
print(len(data_total))

Convert the list of pairs into the appropriate format Transform the data into a Dataset Save dataset on the disk

In [ ]:
formatted_data = {'conversations': data_total}
dataset = Dataset.from_dict(formatted_data)

print(dataset)
print(dataset[0])

dataset.save_to_disk('../files/train_questions_dataset_4000_questions')

## Dataset with 4000 questions: answer label

## Dataset with 4000 questions: short answer label